<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Twitter- Generate leads from post likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_comments_from_post.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #twitter #post #comments #naas_drivers #automation

**Author:** [Alok Chilka](https://www.linkedin.com/in/calok64/)

## Input

### Import libraries

In [ ]:
!pip install --user tweepy

In [ ]:
from naas_drivers import linkedin, hubspot
import pandas as pd
import numpy as np
import naas
from datetime import datetime, timedelta
import requests
import json
import tweepy

### Setup your Twitter

In [ ]:
#Twitter cookies
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAFGZZgEAAAAAyxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

client = tweepy.Client(bearer_token=bearer_token)
user_name = "<YOUR_USERNAME>"
users = client.get_users(usernames=[user_name])
user_id = users.data[0]["id"]


In [ ]:
user_id

### Setup your email to receive notifications

In [7]:
SEND_EMAIL_TO = "<YOUR_EMAIL_ID>"

## Model

### Get posts from Twitter feed

In [ ]:
#Get user tweets
tweets = client.get_users_tweets(id=user_id)

### Build tweet id list

In [ ]:
tweetIdList = []
for tweet in tweets.data:
    tweetIdList.append(str(tweet["id"]))

### Get likes from Twitter posts

In [ ]:
def get_post_stats(tweetIdList):
    for tweetId in tweetIdList:

        print("*********************************")
        print("Tweet ID : ", tweetId)
        parent_tweet = client.get_tweet(id=tweetId, tweet_fields=["author_id", "in_reply_to_user_id",
                                                                             "referenced_tweets,source,public_metrics"])
        #parent_tweet_id = reply_tweet.data["data"]["referenced_tweets"][0]["id"]
        parent_tweet_text = parent_tweet.data["text"]
        parent_public_metrics = parent_tweet.data["data"]["public_metrics"]
        parent_retweet_count = parent_public_metrics["retweet_count"]
        parent_reply_count = parent_public_metrics["reply_count"]
        parent_like_count = parent_public_metrics["like_count"]
        parent_quote_count = parent_public_metrics["quote_count"]
        parent_comments_mentions = []

        print("Tweet Text : ", parent_tweet_text)
        print("Retweet Count : ", parent_retweet_count)
        retweeted_by_users = client.get_retweeters(id=tweetId)
        
        retweeted_by_username = []
        for retweet_user in retweeted_by_users.data:
            retweeted_by_username.append(retweet_user["username"])
        print("Retweeted By : ",retweeted_by_username)
        print("Like Count : ", parent_like_count)
        like_by_users = client.get_liking_users(id=tweetId)
        liked_by_username = []
        for like_user in like_by_users.data:
            liked_by_username.append(like_user["username"])
        print("Liked By : ", liked_by_username)
        print("Reply Count : ", parent_reply_count)
        mentions = client.get_users_mentions(id=user_id)
        replied_by_users = []
        comments = []
        for mention in mentions.data:
            child_tweet = client.get_tweet(id=mention["id"], tweet_fields=["author_id", "in_reply_to_user_id",
                                                                                 "referenced_tweets,source,public_metrics"])
            in_reply_to_tweet_id = child_tweet.data["data"]["referenced_tweets"][0]["id"]

            if str(in_reply_to_tweet_id) == tweetId:
                username = "@"+user_name
                temp = str(mention["text"])
                temp = temp.replace(username,"")
                comments.append(temp)
        print("Comments : ", comments)

        print("Quote Count : ", parent_quote_count)
        print(" ")


## Model

In [ ]:
get_post_stats(tweetIdList)